In [28]:
import sys

sys.path.append("/home/td-user/rtrec")

# Movielens 1M

In [29]:
from rtrec.experiments.datasets import load_dataset

df = load_dataset(name='movielens_1m')

Using existing ratings.dat file.


In [30]:
df[['user', 'item', 'tstamp', 'rating']]

,user,item,tstamp,rating
0,1,1193,978300760,5
1,1,661,978302109,3
2,1,914,978301968,3
3,1,3408,978300275,4
4,1,2355,978824291,5
...,...,...,...,...
1000204,6040,1091,956716541,1
1000205,6040,1094,956704887,5
1000206,6040,562,956704746,5
1000207,6040,1096,956715648,4


In [31]:
# from rtrec.experiments.split import leave_one_last_item
# train_df, test_df = leave_one_last_item(df)

from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df)

#from rtrec.experiments.split import random_split
#train_df, test_df = random_split(df)

In [26]:
train_df

,user,item,rating,tstamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977
...,...,...,...,...
797753,736,1278,4,1045711206
797754,736,3671,4,1045711217
797755,3840,1196,3,1046106127
797756,3840,1225,3,1046106162


In [32]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

model = SLIM()
# nn_feature_selection is fast
#model = SLIM(nn_feature_selection=50) 

recommender = Recommender(model)

In [33]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 3953/3953 [00:52<00:00, 75.28it/s]

Fit completed in 53.68 seconds
Throughput: 14860.85 samples/sec


In [34]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:02<00:00, 27.20it/s]


{'precision': 0.13653973509934242,
 'recall': 0.07255159385029497,
 'f1': 0.07700708957816292,
 'ndcg': 0.15026306640589596,
 'hit_rate': 0.6362582781456954,
 'mrr': 0.2906776253547761,
 'map': 0.07392303311133157,
 'tp': 8247,
 'auc': 0.3380691159466}

In [35]:
recommender.model._recommend(10, top_k=10, filter_interacted=True)

[1073, 1136, 2406, 1035, 2797, 260, 595, 1097, 2987, 3175]

# test partial_fit

In [61]:
from rtrec.experiments.split import temporal_split

test_df1, test_df2 = temporal_split(test_df, test_frac=0.5)

In [62]:
from rtrec.recommender import Recommender
from rtrec.models import SLIM

model = SLIM(nn_feature_selection=50)
recommender = Recommender(model)

In [63]:
recommender.bulk_fit(train_df)

Fitting SLIMElastic: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3953/3953 [00:09<00:00, 406.53it/s]

Fit completed in 11.17 seconds
Throughput: 71397.36 samples/sec


### Note test_df1 is not fit

In [64]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.61it/s]


{'precision': 0.14241342377722274,
 'recall': 0.06721013272032943,
 'f1': 0.07349485624269252,
 'ndcg': 0.15607336413490447,
 'hit_rate': 0.6508389860764013,
 'mrr': 0.3040053722310061,
 'map': 0.07680225795344117,
 'tp': 3989,
 'auc': 0.3518548505919088}

### Fit test_df1

In [65]:
recommender.fit(test_df1, update_interaction=True)

Fitting SLIMElastic: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 3303/3303 [00:09<00:00, 337.54it/s]

Fit completed in 18.79 seconds
Throughput: 5387.52 samples/sec


In [66]:
recommender.evaluate(test_df2, recommend_size=10, filter_interacted=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 24.60it/s]


{'precision': 0.14858978936094297,
 'recall': 0.07101852473705757,
 'f1': 0.07738032259180173,
 'ndcg': 0.16388313081528122,
 'hit_rate': 0.6669046769011068,
 'mrr': 0.3205568589449357,
 'map': 0.08151542919784702,
 'tp': 4162,
 'auc': 0.3622137076894313}